<a href="https://colab.research.google.com/github/andreaFernandezMa/diplomado_de/blob/main/Tareas_Andrea_Fernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark Install

In [1]:
# descomentar esta linea si falla al instalar openjdk
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Hit:14 http://ppa.lau

In [2]:
#Instalacion jdk y descargando spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
#Instalación findspark
!pip install -q findspark
#Seteo de variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [4]:
#Instalar paquete pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 47.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f286c831edc2a2f6f9e9b04ee1e6bcf89033aafebeacb9c374bfaf681a61d015
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


#Crear SparkSession

In [5]:
#Iniciar SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
#Otras Librerias
from pyspark.sql.functions import *


In [7]:
#Conectar google drive con Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# TAREA 1

## Parte 1 (30%)

Utilizando el dataset de [Walmart Stock](https://drive.google.com/file/d/154-RydBa4MOaubvLsiiGtc2HJW1alCkv/view?usp=sharing) realice lo siguiente:

*   Inicie una sesión de Spark
*   Cargue el archivo de Walmart Stock csv desde su cuenta Drive a un dataframe, permita que spark infiera el schema y tipos de datos
*   imprima el schema, si no es correcto, cree usted el schema y cargue de nuevo el archivo.
*   Imprima las primeras 5 filas
*   Relice un *Describe* de la data
*   Cree un nuevo Dataframe con una columna llamada **HV Ratio** el cual tenga la tasa entre **High** versus **Volume** 
*   Determine la media de la columna **Close**
*   Determine el máximo valor de **High** por año.
*   Determine el promedio de **Close** por cada mes.

####Inicie una sesión de Spark

In [8]:
#Inicie una sesión de Spark
#comprobar que la sesion de spark esta ok
spark

####Cargue el archivo de Walmart Stock csv

Se creo el schema debido a que los datos eraan leidos como string

In [9]:
from pyspark.sql.types import (
    DateType,
    FloatType,
    IntegerType,
    StructType,
    StructField, 
    StringType, 
    IntegerType
)

In [10]:
#Creacion de schema
walmart_schema = StructType(
    [
        StructField('Date', DateType(), False),
        StructField('Open',FloatType(), False),
        StructField('High',FloatType(), False),
        StructField('Low',FloatType(), False),
        StructField('Close',FloatType(), False),
        StructField('Volume',IntegerType(), False),
        StructField('Adj Close',FloatType(), False),
   ]
)

In [11]:
#Cargue el archivo de Walmart Stock csv desde su cuenta Drive a un dataframe, permita que spark infiera el schema y tipos de datos
walmart = '/content/tarea1.1/walmart_stock.csv'
#df_walmart = spark.read.csv(walmart, header=True, inferSchema=True) 
df_walmart = spark.read.csv(walmart, header=True, schema=walmart_schema) 

####imprima el schema
Se importaron los formatos que necesitaremos para poder desarrollar el ejercicio

In [12]:
#imprima el schema, si no es correcto, cree usted el schema y cargue de nuevo el archivo.
df_walmart.printSchema() 

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: float (nullable = true)



####Imprima las primeras 5 filas

In [13]:
#Imprima las primeras 5 filas
df_walmart.show(5)

+----------+-----+-----+-----+-----+--------+---------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|
+----------+-----+-----+-----+-----+--------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|
+----------+-----+-----+-----+-----+--------+---------+
only showing top 5 rows



####Relice un Describe de la data

In [14]:
# Relice un Describe de la data
df_walmart.describe()

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

####Cree un nuevo Dataframe 
Con una columna llamada HV Ratio el cual tenga la tasa entre High versus Volume

In [15]:
#Cree un nuevo Dataframe con una columna llamada HV Ratio el cual tenga la tasa entre High versus Volume
df_walmart2 = df_walmart.withColumn('HV', col('High')/col('Volume'))

In [16]:
df_walmart2.show()

+----------+-----+-----+-----+-----+--------+---------+--------------------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|                  HV|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|4.819714682786927E-6|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|6.290848662516662E-6|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554| 4.66941298944916E-6|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922| 7.36733843444859E-6|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|8.915604814435727E-6|
|2012-01-10|59.43|59.71|58.98|59.04| 6907300| 51.49411|8.644477449144044E-6|
|2012-01-11|59.06|59.53|59.04| 59.4| 6365600|51.808098|9.351828386844425E-6|
|2012-01-12|59.79| 60.0| 59.4| 59.5| 7236400|51.895317| 8.29141562102703E-6|
|2012-01-13|59.18|59.61|59.01|59.54| 7729300|51.930202|7.712212051589609E-6|
|2012-01-17|59.87|60.11|59.52|59.85| 8500000| 52.20058|7.071764777688419...|

####Determine la media de la columna Close

In [17]:
#Determine la media de la columna Close
df_walmart.select(avg('Close').alias('promedio_close')).show()

+-----------------+
|   promedio_close|
+-----------------+
|72.38844997363553|
+-----------------+



####Determine el máximo valor de High por año

In [18]:
df_walmart2.groupBy(year(col('Date')).alias('Año')).max('High').show()

+----+---------+
| Año|max(High)|
+----+---------+
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|     77.6|
|2016|    75.19|
+----+---------+



####Determine el promedio de Close por cada mes.

In [19]:
df_walmart2.groupBy(month(col('Date')).alias('Mes')).avg('Close').show()

+---+-----------------+
|Mes|       avg(Close)|
+---+-----------------+
| 12|72.84792482628012|
|  1| 71.4480196131338|
|  6| 72.4953774506191|
|  3|71.77794376266337|
|  5|72.30971685445533|
|  9|72.18411782208611|
|  4|72.97361900692894|
|  8| 73.0298185521906|
|  7|74.43971944078106|
| 10| 71.5785454489968|
| 11|72.11108927207418|
|  2|71.30680438169499|
+---+-----------------+



## Parte 2 (70%)

Se tiene data de logs en JSON comprimidos que se almacenan cada una hora, se pretende enviar los logs de la hora anterior a la actual a una API, pero es necesario realizar un procesamiento de ellos primero. Los campos de los logs se definen de la siguiente manera:

- `index_name`: correponde a un `string`el cual es un indexador del log y no puede ser `null`.
- `detail`: es un diccionario tipo JSON, su estructura es dinámica y puede variar según el log. No puede ser `null`.
- `timestamp`: corresponde al timestamp en unix epoch del instante en que fué generado el log. Debe ser un entero, pero no siempre llega así, tampoco puede ser `null`.
- `group_by`: campo opcional, el cual corresponde a un string que sirve para agrupar el log.
- `tags`: campo opcional, correponde a un array de strings para permitir hacer indexar los logs en la API y realizar queries más eficientes.

Usted debe procesar los logs en un notebook de python con PySpark y escribir la data resultante de manera que cumpla con lo siguiente:

- El archivo final debe contar con `index_name`, `detail`, `timestamp` y `tags`.
- En caso de que exista `group_by` para un log, debe ser añadido al array de `tags`.
- Si un log no tiene `tags` o es `null`, debe retornar un array vacío.
- Asegurarse de que el campo `timestamp` sea de tipo entero. (Hint: cast y round)
- Debe crear un schema para procesar la data, si bien con grandes volumenes de datos se infiere bien, para pocos logs puede no inferir correctamente.
- Debe guardar la data procesada como archivo tipo JSON particionada en su Google Drive.
- La data debe estar particionada por:
  - `env`: corresponde al ambiente de ejecución (prod y qa)
  - `index`: correpsonde al mismo valor de `index_name`.
  - `group_by`: valor que ya viene en `group_by`, en caso de ser null, asignar valor: `NO_GROUP`
  - `Year`, `Month`, `Day`, `Hour`, donde cada una de esas columnas corresponde a la hora de los logs en el path (hora anterior)

Ejemplo ruta de salida:
`drive/MyDrive/.../env=prod/index=dummy_index/group_by=NO_GROUP/year=2022/month=08/day=17/hour=21/filename.json`

Ejemplo archivo de salida:
```
{
  "index_name" : "dummy_name",
  "detail" : {#some dummy dict}
  "timestamp": 1231231321
  "tags" : ["tag_0",....,"tag_n"]
}
```

Consideraciones:
- Asuma que la data que se encuentra en el path de Googe Drive es de la hora pasada a la actual.
- En la creación de su Schema considere que algunos campos no siempre tendrán la misma estructura, para ello simplemente tratelos como `String`.
- Para el ambiente, simplemente realice la ejecución 2 veces, una con `qa` y otra con `prod`.
- Note que para particionar por fecha debe tomar la hora actual y restarle una hora (libería datetime en python).

####Unir archivos en 1 dataframe
Una vez unida la data podemos tener visualizacion, esto nos ayudará a saber si se debe hacer algun cambio

In [22]:
input_path_order_data = '/content/tarea1.2'
order_df = spark.read.json(input_path_order_data)

In [21]:
order_df.printSchema() 

root
 |-- detail: struct (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- item_uuid: string (nullable = true)
 |    |-- last_date_check: string (nullable = true)
 |    |-- number_of_products: long (nullable = true)
 |    |-- order_uuid: string (nullable = true)
 |    |-- shopper_id: long (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- store_id: long (nullable = true)
 |-- group_by: string (nullable = true)
 |-- index_name: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- timestamp: double (nullable = true)



####Guardar data

In [167]:
output_path = '/content/tarea1.2/output_path'

order_df.write.partitionBy('index_name').mode('append').csv(output_path)


AnalysisException: ignored

In [162]:
#Viendo la cantidad de valores distintos tiene cada columna
#order_df.select('group_by').distinct().show()
order_df.select('index_name').distinct().show()

+-------------------+
|         index_name|
+-------------------+
|     catalog_status|
|             orders|
|shoppers_dispatcher|
+-------------------+



# TAREA 2

In [20]:
df_dog_food = spark.read.option('header', 'true').csv('/content/dog_food.csv')
df_hack_data = spark.read.option('header', 'true').csv('/content/walmart_stock.csv')

In [21]:
df_dog_food.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [22]:
df_hack_data.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

## df_hack_data

In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
labelIndex = StringIndexer(inputCol='Spoiled', outputCol='indexedLabel').fit(df_dog_food)

In [23]:
df_dog_food.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [26]:
featureIndexer = VectorIndexer(inputCol = ['A', 'B', 'C', 'D'], outputCol= 'Spoiled', maxCategories=4)

NameError: ignored

In [23]:
(trainingData, testData) = df_dog_food.randomSplit([0.7,0.3])

In [24]:
rf = RandomForestClassifier(labelCol='indexedLabel', featuresCol='indexedFeatures', numTrees=10)

In [25]:
labelConverter = IndexToString(inputCol='prediction', outputCol='predictedLabel', labels=labelIndex.labels)

In [26]:
pipeline = Pipeline(stages=[labelIndex, featureIndexer, rf, labelConverter])

In [29]:
#model = pipeline.fit(trainingData)


predictions = ModelRF.transform(testData)
predictions.printSchema()
predictions.show(5)

NameError: ignored

##df_hack_data

In [ ]:
spark.stop()